# Assignment

## The task is to define and train a machine learning model for predicting the price of a laptop (`buynow_price` column in the dataset) based on its attributes. When testing and comparing the models, the aim to minimize the RMSE measure